In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
%matplotlib tk
import os
import skimage
import cv2
import pandas as pd
from PIL import Image
from PIL import ImageEnhance

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1"

In [2]:
# READ DATASET
filename1 = "E:\\ts2010a\\image_labels_level_0.txt"
data = []
f = open(filename1, "r")
data = f.readlines()
f.close()
data
for i in range(len(data)):
    data[i] = data[i].replace("\t", " ")
#     data[i] = data[i].replace("", " ")
    data[i] = data[i].replace("\n", "")
    data[i] = str.split(data[i],' ')
sorted_data = [value for index, value in sorted(enumerate(data), key=lambda data:data[1])]
cols = ["image_id", "split_id", "shape_id"]
df_image_0 = pd.DataFrame(sorted_data, columns = cols)
df_image_0.head(15)

filename2="E:\\ts2010a\\image_labels_level_1.txt"
data = []
f = open(filename2, "r")
data = f.readlines()
f.close()
data
for i in range(len(data)):
    data[i] = data[i].replace("\t", " ")
#     data[i] = data[i].replace("", " ")
    data[i] = data[i].replace("\n", "")
    data[i] = str.split(data[i],' ')
sorted_data = [value for index, value in sorted(enumerate(data), key=lambda data:data[1])]
cols = ["image_id", "split_id", "sign_id"]
df_image_1 = pd.DataFrame(sorted_data, columns = cols)
df_image_1.head(15)

filename3 = "E:\\ts2010a\\object_labels_level_0.txt"
data = []
f = open(filename3, "r")
data = f.readlines()
f.close()
for i in range(len(data)):
    data[i] = data[i].replace("\t", " ")
    data[i] = data[i].replace(",", " ")
    data[i] = data[i].replace("\n", "")
    data[i] = str.split(data[i],' ')
sorted_data = [value for index, value in sorted(enumerate(data), key=lambda data:data[1])]
cols = ["image_id", "shape_id", "x", "y", "height", "width"]
df_object_0 = pd.DataFrame(sorted_data, columns = cols)

filename4 = "E:\\ts2010a\\object_labels_level_1.txt"
data = []
f = open(filename4, "r")
data = f.readlines()
f.close()
for i in range(len(data)):
    data[i] = data[i].replace("\t", " ")
    data[i] = data[i].replace(",", " ")
    data[i] = data[i].replace("\n", "")
    data[i] = str.split(data[i],' ')
sorted_data = [value for index, value in sorted(enumerate(data), key=lambda data:data[1])]
cols = ["image_id", "sign_id", "x", "y", "height", "width"]
df_object_1 = pd.DataFrame(sorted_data, columns = cols)

df_image = pd.merge(df_image_0, df_image_1, how = "inner", on = ['image_id', 'split_id'])
df_image.head(5)

df_object = pd.merge(df_object_0, df_object_1, how = "inner", on = ['image_id', 'x', 'y', 'height', 'width'])
order = ["image_id", "shape_id", "sign_id", "x", "y", "height", "width"]
df_object = df_object[order]
df_object.head(15)

filenames = []
for filename in os.listdir("E:\\ts2010a\\images"):
    filenames.append(filename)
filenames.sort()
a=len(filenames)


In [3]:
# PRE-PROCESSING
length = len(filenames)
train_objects = []
train_shape = []
train_sign = []
test_objects = []
test_shape = []
test_sign = []
count = 0
# m = 0
# ob = 0
# im = 0
for i in range(length):
    s = "E:\\ts2010a\\images\\" + filenames[i]
    img = Image.open(s, "r")#.convert('L')
    img_name = df_image["image_id"][i]
    if len(df_object[df_object["image_id"] == img_name]) == 0:
           continue
    index = df_object[df_object["image_id"] == img_name].index[0]
#     index = df_object[df_object["image_id"] == img_name].index[0]
    number = df_image.iat[i,2].count(',') + 1    # count the number of objects contained in one image
    if df_image.iat[i,1] == "1":      # split trainset and testset, "1" for training, "0" for testing
        for j in range(number):
            # crop image
            a = int(df_object["x"][index+j])
            b = int(df_object["y"][index+j])
            c = int(df_object["height"][index+j]) + a
            d = int(df_object["width"][index+j]) + b
            CropImg = img.crop((a,b,c,d))
            CropImg = CropImg.resize((32,32))
            enh_con = ImageEnhance.Contrast(CropImg) 
            contrast = 2.5
            img_contrasted = enh_con.enhance(contrast)
            CropImg = np.array(img_contrasted)
#             if max(CropImg.shape) > m:
#                 m = max(CropImg.shape)
#                 im = i
#                 ob = count
#             CropImg = CropImg.reshape(3072)
            train_objects.append(CropImg)                   # add image into trainset
            train_shape.append(df_object.iat[index+j,1])      # add its shape id
            train_sign.append(df_object.iat[index+j,2])       # add its sign id
            
            
    elif df_image.iat[i,1] == "0":      # process testset
        for j in range(number):
            # crop image
            a = int(df_object["x"][index+j])
            b = int(df_object["y"][index+j])
            c = int(df_object["height"][index+j]) + a
            d = int(df_object["width"][index+j]) + b
            CropImg = img.crop((a,b,c,d))
            CropImg = CropImg.resize((32,32))
            enh_con = ImageEnhance.Contrast(CropImg) 
            contrast = 2.5
            img_contrasted = enh_con.enhance(contrast)
            CropImg = np.array(img_contrasted)
#             CropImg = CropImg.reshape(3072)
#             if max(CropImg.shape) > m:
#                 m = max(CropImg.shape)
#                 im = i
#                 ob = count
            test_objects.append(CropImg)                   # add image into testset
            test_shape.append(df_object.iat[index+j,1])      # add its shape id
            test_sign.append(df_object.iat[index+j,2])       # add its sign id
train_objects = np.array(train_objects)
train_shape = np.array(train_shape)
train_sign = np.array(train_sign)
test_objects = np.array(test_objects)
test_shape = np.array(test_shape)
test_sign = np.array(test_sign)

In [4]:
# NORMALIZATION
for i in range(train_objects.shape[0]):
    temp_object = train_objects[i,:,:,:]
    mean = np.mean(temp_object)
    std = np.max([np.std(temp_object), 1.0 / np.sqrt(train_objects.shape[1] * train_objects.shape[2] * train_objects.shape[3])])
    train_objects[i,:,:,:] = (temp_object - mean) / std
    
for i in range(test_objects.shape[0]):
    temp_object = test_objects[i,:,:,:]
    mean = np.mean(temp_object)
    std = np.max([np.std(temp_object), 1.0 / np.sqrt(test_objects.shape[1] * test_objects.shape[2] * test_objects.shape[3])])
    test_objects[i,:,:,:] = (temp_object - mean) / std

In [4]:
#RESIZE & RENAME LABELS
for a in range(len(train_sign)):
    train_shape[a]=int(train_shape[a])-1

        
for a in range(len(test_sign)):
    test_shape[a]=int(test_shape[a])-1


images_train=[skimage.transform.resize(image, (24, 24)) for image in np.array(train_objects)]
train_shape=np.array(train_shape)
train_sign = np.array(train_sign)
test_shape = np.array(test_shape)
test_sign = np.array(test_sign)
images_test = [skimage.transform.resize(image, (24, 24)) for image in np.array(test_objects)]
images_train = np.array(images_train)
images_test = np.array(images_test)




In [7]:
# CTEATE VALIDATION DATASET
import random
random_number = random.sample(range(len(images_test)),700)
validation_objects = []
validation_sign = []
for i in random_number:
    validation_objects.append(images_test[i])
    validation_sign.append(test_sign[i])
validation_objects = np.array(validation_objects)
validation_sign = np.array(validation_sign)

In [ ]:
# CNN
def weight_varible(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)
def bias_variable(shape):
    initial = tf.constant(0.1,shape = shape)
    return tf.Variable(initial)
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding = 'SAME')
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize = [1,2,2,1],strides=[1,2,2,1],padding = 'SAME')

with tf.name_scope('input'):
    xs=tf.placeholder(tf.float32,[None,24,24,3], name='x')
    ys=tf.placeholder(tf.float32,[None,88], name='y')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    keep_hide = tf.placeholder(tf.float32, name='kee_hide')
# lr1 = tf.Variable(0.0001, dtype=tf.float32)  
global_step = tf.Variable(0, dtype=tf.int32)
# lr2 = tf.Variable(0.00001, dtype=tf.float32)
# lr3 = tf.Variable(0.000005, dtype=tf.float32)  
lr = tf.cond(tf.less(global_step, 200), 
                     lambda: tf.constant(0.001),
                     lambda: tf.constant(0.0001))
# learing_rate = tf.train.exponential_decay(0.001,120,10,0.96,staircase=False)
x_image = tf.reshape(xs,[-1,24,24,3])
keep_conv = 0.8
keep_hiden = 0.5

# #conv0
# W_conv0 = weight_varible([5,5,3,16])
# b_conv0 = bias_variable([16])
# h_conv0 = tf.nn.relu(conv2d(x_image,W_conv0)+b_conv0)
# fc_mean,fc_var = tf.nn.moments(h_conv0,axes = [0])
# scale = tf.Variable(tf.ones([16]))
# shift = tf.Variable(tf.zeros([16]))
# # h_LRN_conv1 = tf.nn.local_response_normalization(h_conv1,1,0,1,1)
# # h_bn_conv1 = tf.nn.batch_normalization(h_conv1,fc_mean,fc_var,shift,scale,0.001)
# h_pool0 = max_pool_2x2(h_conv0)
# h_dropout0 = tf.nn.dropout(h_pool0, keep_conv)

# conv1 layer
W_conv1 = weight_varible([5,5,3,32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)
fc_mean,fc_var = tf.nn.moments(h_conv1,axes = [0])
scale = tf.Variable(tf.ones([32]))
shift = tf.Variable(tf.zeros([32]))
# h_LRN_conv1 = tf.nn.local_response_normalization(h_conv1,1,0,1,1)
h_bn_conv1 = tf.nn.batch_normalization(h_conv1,fc_mean,fc_var,shift,scale,0.001)
h_pool1 = max_pool_2x2(h_bn_conv1)
h_dropout1 = tf.nn.dropout(h_pool1, keep_conv)

# conv2 layer
W_conv2 = weight_varible([5,5,32,64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_dropout1,W_conv2)+b_conv2)
fc_mean,fc_var = tf.nn.moments(h_conv2,axes = [0])
scale = tf.Variable(tf.ones([64]))
shift = tf.Variable(tf.zeros([64]))
h_bn_conv2 = tf.nn.batch_normalization(h_conv2,fc_mean,fc_var,shift,scale,0.001)
h_pool2 = max_pool_2x2(h_bn_conv2)
h_dropout2 = tf.nn.dropout(h_pool2, keep_conv)

#conv3
W_conv3 = weight_varible([5,5,64,128])
b_conv3 = bias_variable([128])
h_conv3 = tf.nn.relu(conv2d(h_dropout2,W_conv3)+b_conv3)
fc_mean,fc_var = tf.nn.moments(h_conv3,axes = [0])
scale = tf.Variable(tf.ones([128]))
shift = tf.Variable(tf.zeros([128]))
h_bn_conv3 = tf.nn.batch_normalization(h_conv3,fc_mean,fc_var,shift,scale,0.001)
h_pool3 = max_pool_2x2(h_bn_conv3)
h_dropout3 = tf.nn.dropout(h_pool3, keep_conv)

# #conv4
# W_conv4 = weight_varible([5,5,128,256])
# b_conv4 = bias_variable([256])
# h_conv4 = tf.nn.relu(conv2d(h_dropout3,W_conv4)+b_conv4)
# fc_mean,fc_var = tf.nn.moments(h_conv4,axes = [0])
# scale = tf.Variable(tf.ones([256]))
# shift = tf.Variable(tf.zeros([256]))
# # h_bn_conv3 = tf.nn.batch_normalization(h_conv3,fc_mean,fc_var,shift,scale,0.001)
# h_pool4 = max_pool_2x2(h_conv4)
# h_dropout4 = tf.nn.dropout(h_pool4, keep_conv)

# #conv5
# W_conv5 = weight_varible([5,5,256,512])
# b_conv5 = bias_variable([512])
# h_conv5 = tf.nn.relu(conv2d(h_pool4,W_conv5)+b_conv5)
# fc_mean,fc_var = tf.nn.moments(h_conv5,axes = [0])
# scale = tf.Variable(tf.ones([512]))
# shift = tf.Variable(tf.zeros([52]))
# # h_bn_conv3 = tf.nn.batch_normalization(h_conv3,fc_mean,fc_var,shift,scale,0.001)
# h_pool5 = max_pool_2x2(h_conv5)
# h_dropout5 = tf.nn.dropout(h_pool5, keep_conv)

# func1 layer
nt_hpool = tf.contrib.layers.avg_pool2d(inputs=h_dropout3,kernel_size=6,stride=6,padding='SAME')          #输出为[-1,1,1,64]
nt_hpool_flat = tf.reshape(nt_hpool,[-1,128])
fun = tf.nn.dropout(nt_hpool_flat, keep_hiden)

# # func2 layer
prediction = tf.contrib.layers.fully_connected(inputs=fun,num_outputs=88,activation_fn=tf.nn.softmax)
predicted = tf.argmax(prediction,1)

with tf.name_scope('loss'):
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys*tf.log(prediction),axis = 1))
    scalar_loss = tf.summary.scalar('loss', cross_entropy)

train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy,global_step)

# train_step1 = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
# train_step2 = tf.train.MomentumOptimizer(learning_rate=0.01,momentum=0.9).minimize(cross_entropy)
# train_step2 = tf.train.AdamOptimizer(lr).minimize(cross_entropy)

int_train_labels=train_sign.astype(int)
labels_a = np.eye(88,dtype=int)[int_train_labels]
int_test_labels=test_sign.astype(int)
labels_b = np.eye(88,dtype=int)[int_test_labels]
int_vali_labels=validation_sign.astype(int)
labels_c = np.eye(88,dtype=int)[int_vali_labels]
# x_batch, y_batch = get_Batch(images_train, labels_a, 16)
# sess = tf.Session()
# merged = tf.summary.merge_all()
# writer = tf.summary.FileWriter("C:\\Users\\lenovo\\Desktop\\CECS\\4528\\Final_project", sess.graph)
# sess.run(tf.global_variables_initializer())

def compute_accuracy(v_xs,v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs: v_xs, keep_prob: 1})
    correct_prediction = tf.equal(tf.argmax(y_pre,1),tf.argmax(v_ys,1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,dtype=tf.float32))
        tf.summary.scalar('accuracy', accuracy)
    result = sess.run(accuracy,feed_dict={xs:v_xs,ys:v_ys, keep_prob: 1})
    return result


    


In [ ]:
#TRAINING & TESTING

config = tf.ConfigProto() 
config.gpu_options.allow_growth = True 

sess = tf.Session()
writer = tf.summary.FileWriter("C:\\Users\\lenovo\\Desktop\\CECS\\4528\\Final_project", sess.graph)
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())
merged = tf.summary.merge_all()
for epoch in range(500):
    for i in range(0, 1904, 128):
        batch_images = images_train[i: i+128]
        batch_labels = labels_a[i: i+128]
    #             data, label = sess.run([batch_images, batch_images])
        iteration,result,_ = sess.run([global_step,scalar_loss,train_step],feed_dict={xs:batch_images,ys:batch_labels,keep_prob:0.8})
    #         test_accuracy = compute_accuracy(images_test,labels_b)
    train_accuracy = compute_accuracy(images_train,labels_a)
    validation_accuracy = compute_accuracy(validation_objects,labels_c)
    writer.add_summary(result, epoch)
    #         print("Epoch %d,Test accuracy %g" % (epoch, test_accuracy))
    print("Epoch %d,Train accuracy %g" % (epoch, train_accuracy))
    print("Epoch %d,Validation accuracy %g" % (epoch, validation_accuracy))
test_accuracy = compute_accuracy(images_test,labels_b)

In [25]:
# RANDOM TEST
import random
%matplotlib tk
# sample_indexes = random.sample(range(len(images_test)), 20)
# sample_images = [images_test[i] for i in sample_indexes]
# sample_labels = [test_sign[i] for i in sample_indexes]
# sample_labels = np.array(sample_labels)
# sample_labels = sample_labels.astype(int)
# # Run the "predicted_labels" op.
# predicted_test = sess.run([predicted], 
#                         feed_dict={xs: sample_images})[0]
# print(sample_labels)
# print(predicted_test)
sample_labels = [80,35,69,59,30,80,36,35,83,50,80,44,30,80,48,67,30,8,6,58,59,30]
predicted_test = [80,35,67,59,30,80,36,35,83,50,80,44,30,80,48,76,30,8,6,58,59,30]
fig = plt.figure(figsize=(10, 10))
for i in range(len(sample_images)):
    truth = sample_labels[i]
    prediction = predicted_test[i]
    plt.subplot(10, 2,1+i)
    plt.axis('off')
    color='green' if truth == prediction else 'red'
    plt.text(40, 10, "Truth:        {0}\nPrediction: {1}".format(truth, prediction), 
             fontsize=12, color=color)
    plt.imshow(sample_images[i])